## Ensaio tribológico - Aquisição e Preparação dos Dados de Coeficiente de atrito

O sinal adquirido pelo computador ligado ao tribômetro é registrado em um arquivo de extensão `.tst`. Este arquivo é aberto e exportado para `.csv` por um software do mesmo fabricante do tribômetro, denominado *Viewer*. Deste modo, cada ensaio gerou um arquivo `nome_do_ensaio.csv` com os dados de tempo, força normal, força de atrito e demais informações gravadas do tribômetro ou pré-processadas pelo software.

Deste modo, foi necessária a aquisição dos dados individuais e preparação de uma base de dados com os valores médios e de dispersão dos valores de coeficiente de atrito, que são de central importância para este trabalho. 

#### Definição da Classe Test

Iniciamos com a definição de uma classe para agrupar todos os dados referentes a um ensaio, que chamamos de classe `Test`. Para esta etapa, usaremos as bibliotecas `pandas`, `numpy` e `csv`.

In [1]:
import pandas as pd
import numpy as np
import csv
pd.options.plotting.backend = 'plotly'
import plotly.graph_objects as go

Neste documento, utilizaremos uma versão simplificada da classe `Test`, mostrando seus métodos mais importantes e os dados que ela guarda.

In [2]:
class Test(object):
    '''
    A class to represent a tribology test
    [...]
    '''


    def __init__ (self,name='default',method=None,stroke=4.66,df=pd.DataFrame()): #o ensaio é inicializado com nome default, freq None e dataframe vazio
        self.name = name
        self.f = 0
        self.F = 0
        self.method = method
        self.data = df
        self.stroke = stroke

            
    def __str__ (self):
        try:
            string = """Ensaio: {}
Método: Reciprocating
COF: {:.4f} +/- {:.4f}
Força Normal: {:.1f} N
Frequência: {:.1f} Hz
Distância Total de Deslizamento: {:.1f} mm
Stroke: {:.2f} mm
            """.format(self.name,
            float(self.meanCOF),
            float(self.stdCOF),
            float(self.getForce()),
            float(self.getSpeed()),
            float(self.getDistance()),
            float(4.66)
            )
        except AttributeError:
            string = 'Ensaio vazio'
        return string

    
    def __repr__ (self):
        return str(self)


    def setName(self,name): #setter para string com local/nome do arquivo original
        self.name = name


    def setTimestamp(self, timestamp): #setter para a timestamp do início do ensaio
        self.timestamp = timestamp


    def setSpeed(self,f): #setter para a frequência do ensaio
        self.f = f


    def setForce(self,F): #setter para a força do ensaio
        self.F = F


    def setData(self,dataseries): #setter para o dataframe de dados do ensaio
        #dados originais, considerando apenas tempo, Força Normal e COF
        self.data = dataseries.filter(items=['T','Fz','COF'])
        #encontra onde está o COF=0 (reversão de velocidade)
        self.zero = self.data.tail(n = self.getSize()//2).nsmallest(1,'COF').iat[0,0]
        instant_position = 4.66/2 * np.cos(2.*np.pi*self.f*(self.data['T']-self.zero))
        instant_speed = - 4.66 * np.pi * self.f * np.sin((2.*np.pi*self.f*(self.data['T']- self.zero)))
        self.data = self.data.assign(V = instant_speed, X = instant_position)
        #atribui duas novas colunas: X, com a posição em relação ao MHS.
                                    #V, com a velocidade instantânea de deslizamento
        self.meanCOF = self.getHighCOFstat().at['mean','COF'] #Calcula e salva o cof médio para o ensaio
        self.stdCOF = self.getHighCOFstat().at['std','COF'] #Calcula e salva o desvio padrão do cof para o ensaio
        self.meanCOF_static = self.getHighCOFstat_static().at['mean','COF']
        self.stdCOF_static = self.getHighCOFstat_static().at['std','COF']


    def getName(self): #retorna o local/nome do arquivo do ensaio
        return self.name


    def getTimestamp(self):
        return self.timestamp


    def getSpeed(self): #retorna um float com a frequência usada no ensaio
        return self.f


    def getForce(self): #retorna um float com a força usada no ensaio
        return self.F


    def getData(self): #retorna um dataframe com os dados de ensaio
        return self.data


    def getCOF(self): #retorna uma lista com [COF médio, desv pad do COF]
        COF = [self.meanCOF,self.stdCOF]
        return COF


    def getSize(self): #retorna um int do número de pontos do ensaio
        size = self.data.shape[0]
        return size


    def getDuration(self): #retorna um float da duração do ensaio em segundos
        duration = self.data['T'].max()
        return duration


    def getDistance(self): #retorna um float da distância, em milímetros, total do ensaio
        d = self.getSpeed() * 2 * 4.66 * (self.getDuration())
        return d


    def getCOFe(self): #retorna um float do coeficiente de atrito energético, como calculado por Vale, Silva e Pintaúde (2019)
        self.data = self.data.assign(COFe = self.data['COF']*(abs(self.data['V'])*0.001))
        return float(self.data['COFe'].sum()/self.getDistance())


    def getVmed(self): #retorna um float da velocidade média do ensaio em milímetros por segundo
        Vmed = round(self.getDistance() / self.getDuration(), 3)
        return Vmed


    def getupperdata(self): #retorna um dataframe dos pontos acima da velocidade média (movimento de ida)
        ud = self.data[self.data.V >= self.getVmed()]
        return ud


    def getlowerdata(self): #retorna um dataframe dos pontos abaixo da velocidade média negativa (movimento de volta)
        ld = self.data[self.data.V <= -self.getVmed()]
        return ld


    def getHighCOFdata(self, p): #retorna um dataframe dos pontos cujo coeficiente de atrito médio é mais alto
        A = self.getupperdata()[self.getupperdata()['T']>=(1-p)*float(self.getDuration())]
        B = self.getlowerdata()[self.getlowerdata()['T']>=(1-p)*float(self.getDuration())]
        if (A.describe().at['mean','COF']>=B.describe().at['mean','COF']):
            return A
        else:
            return B

        
    def getupperdata_static(self): #retorna um dataframe dos pontos acima da velocidade média
        ud = self.data[(self.data.V <= self.getVmed())&(self.data.V>0)]
        return ud


    def getlowerdata_static(self): #retorna um dataframe dos pontos abaixo da velocidade média negativa
        ld = self.data[(self.data.V >= -self.getVmed())&(self.data.V<0)]
        return ld


    def getHighCOFdata_static(self, p): #retorna um dataframe dos pontos cujo coeficiente de atrito médio é mais alto
        A = self.getupperdata_static()[self.getupperdata_static()['T']>=(1-p)*float(self.getDuration())]
        B = self.getlowerdata_static()[self.getlowerdata_static()['T']>=(1-p)*float(self.getDuration())]
        if (A.describe().at['mean','COF']>=B.describe().at['mean','COF']):
            return A
        else:
            return B


    def getHighCOFstat(self,p=0.25): #retorna um dataframe das estatísticas dos pontos cujo coeficiente de atrito médio é mais alto
        return self.getHighCOFdata(p).describe()


    def getHighCOFstat_static(self,p=0.25): #retorna um dataframe das estatísticas dos pontos cujo coeficiente de atrito médio é mais alto
        return self.getHighCOFdata_static(p).describe()

Deve-se notar que a Classe Test retorna alguns tipos de coeficiente de atrito, quais sejam:

- **Coeficiente de atrito cinético**: com a função `getHighCOFdata`, é obtido o coeficiente de atrito cinético dos pontos onde a velocidade instantânea é superior, em módulo, à velocidade média, em uma das direções de movimento (ida ou volta). A direção escolhida é aquela em que se observou o maior coeficiente de atrito cinético médio. Ao obter a média e desvio-padrão, somente é considerada a porção final do ensaio, correspondendo aos 25% finais ou 7,5m de deslizamento.
- **Coeficiente de atrito quasi-estático**: com a função `getHighCOFdata_static`, é obtido o coeficiente de atrito na região onde a velocidade instantânea é mais baixa, em módulo, que a velocidade média. Este valor não foi utilizado nas próximas etapas de análise.
- **Coeficiente de atrito energético**: obtido com a função `getCOFe`, o coeficiente de atrito energético é uma média ponderada do atrito com a velocidade instantânea. Ou seja, valores de atrito nas posições de maior velocidade tem maior peso, enquanto aqueles nas regiões de menor velocidade têm menor peso. Essa metodologia foi desenvolvida por Vale, Silva e Pintaude (2019) no artigo [*Energetic coefficient of friction applied to cylinder liners lab tests*](https://doi.org/10.1108/ILT-08-2019-0324).

### Utilizando a Classe Test

Para utilizar a Classe criada, podemos inicializar um ensaio vazio.

In [3]:
new_test = Test()

In [4]:
new_test

Ensaio vazio

Agora podemos adicionar os dados ao teste. Precisaremos de quatro informações obtidas através do `.csv`: os dados de atrito, a velocidade utilizada no ensaio (frequência de oscilação), a força normal aplicada e o tempo do início do ensaio. Para isso, quatro funções são definidas.

In [5]:
def findData(filename): #abre o CSV para recuperar os dados de ensaio, ignorando o cabeçalho
    df = pd.read_csv(filename,
                     header=17,
                     skiprows = [21],
                     dtype = np.float64)
    return df

def findSpeed(filename): #abre o CSV para recuperar a frequência/velocidade do ensaio a partir do cabeçalho
    df = pd.read_csv(filename,
                     sep='\n',
                     skiprows=13,
                     nrows=5,
                     names=["Dados"],
                     dtype = 'string')
    linha = df.at[1, "Dados"]
    l = linha.replace(' ','')
    ldic = locals()
    exec(l, globals(), ldic)
    Velocity = ldic["Velocity"]
    return float(Velocity)


def findForce(filename): #abre o CSV para recuperar a força do ensaio a partir do cabeçalho
    df = pd.read_csv(filename,
                     sep='\n',
                     skiprows=13,
                     nrows=5,
                     names=["Dados"],
                     dtype = 'string')
    linha = df.at[2, "Dados"]
    l = linha.replace(' ','')
    ldic = locals()
    exec(l, globals(), ldic)
    F = ldic["SetForce"]
    return float(-F)


def findTimestamp(filename): #abre o CSV para recuperar tempo de início do ensaio a partir do cabeçalho
    df = pd.read_csv(filename,
                     sep=' date ',
                     skiprows=9,
                     nrows=1,
                     names=['trash','timestamp'],
                     engine = 'python',
                     dtype = 'string')
    time = df.at[0,'timestamp']
    timestamp = pd.Timestamp(time)
    return timestamp

In [6]:
filename = 'src/PU_atrito/Umidade_FC_001.zip'
new_test.setSpeed(findSpeed(filename))
new_test.setForce(findForce(filename))
new_test.setData(findData(filename))
new_test.setName(filename)
new_test.setTimestamp(findTimestamp(filename))

O teste `new_test` foi populado com informações. Ele é representado por uma string que resume os principais dados contidos. Podemos usar a função `new_test.getData()` para visualizar a tabela de dados obtidas do arquivo `.csv`.

In [7]:
new_test

Ensaio: src/PU_atrito/Umidade_FC_001.zip
Método: Reciprocating
COF: 0.3919 +/- 0.0184
Força Normal: 2.5 N
Frequência: 4.0 Hz
Distância Total de Deslizamento: 30161.3 mm
Stroke: 4.66 mm
            

In [8]:
new_test.getData()

,T,Fz,COF,V,X
0,0.000,-2.495,0.005,-50.957340,-1.148055
1,0.001,-2.525,0.005,-50.216150,-1.198644
2,0.002,-2.549,0.005,-49.443241,-1.248476
3,0.003,-2.571,0.007,-48.639104,-1.297520
4,0.004,-2.585,0.014,-47.804245,-1.345744
...,...,...,...,...,...
809044,809.044,-2.515,0.307,2.942271,-2.327057
809045,809.045,-2.544,0.342,4.411084,-2.323380
809046,809.046,-2.570,0.364,5.877110,-2.318236
809047,809.047,-2.582,0.350,7.339425,-2.311627


Em seguida, foi criada uma função para criar uma lista de testes rapidamente, executando as funções descritas acima em uma série de arquivos sequenciais.

In [9]:
def createTestList(metodo,lista_arquivos): #abre os CSVs e armazena numa lista de ensaios
    '''
    Creates a list of Tests from .csv files
    [...]
    '''
    testlist = []
    for filename in lista_arquivos:
        test_i = Test(method = metodo)
        test_i.setSpeed(findSpeed(filename))
        test_i.setForce(findForce(filename))
        test_i.setData(findData(filename))
        test_i.setName(filename)
        test_i.setTimestamp(findTimestamp(filename))
        testlist.append(test_i)
    return testlist

Podemos, agora, executar o seguinte código para obter os 16 testes referentes aos 16 ensaios realizados com PU Puro e umidade controlada.

In [10]:
filenames = [
    'src/PU_atrito/Umidade_FC_001.zip',
    'src/PU_atrito/Umidade_FC_002.zip',
    'src/PU_atrito/Umidade_FC_003.zip',
    'src/PU_atrito/Umidade_FC_004.zip',
    'src/PU_atrito/Umidade_FC_005.zip',
    'src/PU_atrito/Umidade_FC_006.zip',
    'src/PU_atrito/Umidade_FC_007.zip',
    'src/PU_atrito/Umidade_FC_008.zip'
]
lista_testes  = createTestList('Reciprocating', filenames)

In [11]:
lista_testes

[Ensaio: src/PU_atrito/Umidade_FC_001.zip
 Método: Reciprocating
 COF: 0.3919 +/- 0.0184
 Força Normal: 2.5 N
 Frequência: 4.0 Hz
 Distância Total de Deslizamento: 30161.3 mm
 Stroke: 4.66 mm
             ,
 Ensaio: src/PU_atrito/Umidade_FC_002.zip
 Método: Reciprocating
 COF: 0.3322 +/- 0.0139
 Força Normal: 5.0 N
 Frequência: 4.0 Hz
 Distância Total de Deslizamento: 30162.2 mm
 Stroke: 4.66 mm
             ,
 Ensaio: src/PU_atrito/Umidade_FC_003.zip
 Método: Reciprocating
 COF: 0.4126 +/- 0.0172
 Força Normal: 2.5 N
 Frequência: 4.0 Hz
 Distância Total de Deslizamento: 30163.2 mm
 Stroke: 4.66 mm
             ,
 Ensaio: src/PU_atrito/Umidade_FC_004.zip
 Método: Reciprocating
 COF: 0.2872 +/- 0.0093
 Força Normal: 8.0 N
 Frequência: 4.0 Hz
 Distância Total de Deslizamento: 30164.1 mm
 Stroke: 4.66 mm
             ,
 Ensaio: src/PU_atrito/Umidade_FC_005.zip
 Método: Reciprocating
 COF: 0.3245 +/- 0.0156
 Força Normal: 5.0 N
 Frequência: 2.0 Hz
 Distância Total de Deslizamento: 30104.5 

Com isso, podemos reunir os principais dados em um dataframe, possibilitando a posterior análise destes dados em outra etapa. Para isso criaremos um novo dataframe com a função `pd.DataFrame()` e iremos montar os dados na forma longa. Ou seja, cada coluna corresponderá a um parâmetro e cada linha corresponderá a um ensaio.

In [12]:
material = 'PU Puro'
descricao = 'Umidade Controlada'

df = pd.DataFrame(columns = ["Início",
                             "Material",
                             "Força Normal (N)",
                             "Frequência (Hz)",
                             "Distância (m)",
                             "COF Médio (-)",
                             "COF Desv. Pad. (-)",
                             "COF Médio S (-)",
                             "COF Desv. Pad. S (-)",
                             "COF Energético (-)",
                             'Local'])

In [13]:
for ensaio in lista_testes:
    dic = {
        "Início": ensaio.getTimestamp(),
        "Material": material,
        "Descrição": descricao,
        "Força Normal (N)": ensaio.getForce(),
        "COF Médio (-)": ensaio.getCOF()[0],
        "COF Desv. Pad. (-)": ensaio.getCOF()[1],
        'Local': ensaio.getName(),
        "Distância (m)": ensaio.getDistance()//1000,
        "Frequência (Hz)": ensaio.getSpeed(),
        "COF Energético (-)": ensaio.getCOFe(),
        "COF Médio S (-)": ensaio.meanCOF_static,
        "COF Desv. Pad. S (-)": ensaio.stdCOF_static,
    }
    df = df.append(dic, ignore_index = True)

In [14]:
df

,Início,Material,Força Normal (N),Frequência (Hz),Distância (m),COF Médio (-),COF Desv. Pad. (-),COF Médio S (-),COF Desv. Pad. S (-),COF Energético (-),Local,Descrição
0,2020-11-19 12:26:33,PU Puro,2.50,4.0,30.0,0.391869,0.018417,0.392966,0.081895,0.347091,src/PU_atrito/Umidade_FC_001.zip,Umidade Controlada
1,2020-11-19 13:55:10,PU Puro,5.00,4.0,30.0,0.332230,0.013931,0.339033,0.074096,0.297532,src/PU_atrito/Umidade_FC_002.zip,Umidade Controlada
2,2020-11-19 14:18:49,PU Puro,2.50,4.0,30.0,0.412585,0.017158,0.388892,0.081370,0.334447,src/PU_atrito/Umidade_FC_003.zip,Umidade Controlada
3,2020-11-19 14:47:23,PU Puro,8.00,4.0,30.0,0.287215,0.009275,0.291097,0.080997,0.261241,src/PU_atrito/Umidade_FC_004.zip,Umidade Controlada
4,2020-11-19 15:35:15,PU Puro,5.00,2.0,30.0,0.324453,0.015587,0.335845,0.080062,0.307601,src/PU_atrito/Umidade_FC_005.zip,Umidade Controlada
5,2020-11-19 16:07:05,PU Puro,5.17,3.0,30.0,0.363054,0.014245,0.332598,0.069298,0.310113,src/PU_atrito/Umidade_FC_006.zip,Umidade Controlada
6,2020-11-19 16:33:10,PU Puro,5.17,3.0,30.0,0.335115,0.012771,0.315983,0.077308,0.302822,src/PU_atrito/Umidade_FC_007.zip,Umidade Controlada
7,2020-11-19 16:53:28,PU Puro,5.17,3.0,30.0,0.308932,0.010045,0.312032,0.078581,0.290558,src/PU_atrito/Umidade_FC_008.zip,Umidade Controlada


In [17]:
df.to_csv('src/summary_pu.csv')

O mesmo pode ser feito com os outros materiais e tipos de ensaio, de modo a obter um dataframe completo com todos os ensaios realizados. Este processo foi automatizado em um script, com a utilização da biblioteca `Gooey` para criar uma GUI (*Graphical User Interface*, ou interface gráfica para o usuário) e facilitar o uso do programa. Capturas de tela, abaixo, mostram o programa em funcionamento.

#### Tela Inicial

![Etapa 1](src/tribo_01.png)

#### Seleção do Tipo de Ensaio

O programa está configurado corretamente para ensaios Reciprocating, Pin-on-disk e Scratch Test

![Etapa 2](src/tribo_02.png)

#### Seleção dos arquivos .csv

![Etapa 3](src/tribo_03.png)

#### Outras informações do arquivo

Cada grupo de testes deve ser feito separadamente, ou seja, apenas um material e uma descrição para cada vez que clicar em `Start`.

![Etapa 4](src/tribo_04.png)

#### Rodando

O programa leva alguns minutos para processar os arquivos, dependendo do tamanho e quantidade de ensaios.

![Etapa 5](src/tribo_05.png)

#### Finalizado

Caso haja mais ensaios com outros materiais ou descrições, deve-se clicar em `Edit` e selecionar os novos ensaios, mantendo a pasta e arquivo de saída inalterados.

![Etapa 6](src/tribo_06.png)